# Evaluation Regression Models

In [37]:
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns

from scipy import stats

import warnings
warnings.filterwarnings("ignore")

import sklearn.metrics
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LinearRegression
from sklearn.feature_selection import f_regression
from sklearn.metrics import r2_score

from math import sqrt

Note from Ryan Orsinger:

TSS = ((y - y.mean())**2).sum(), which is the sum of the squares of the differences between the actual y values (since this is supervised learning and we have the target variable) and the baseline, which is y.mean().

ESS = ((y_predicted - y.mean())**2).sum(), which is the sum of the squares of the difference between the model’s prediction and the baseline.

R squared is the ratio of ESS to TSS, or ESS/TSS.
In English, R squared is the percent of y explained by x

and unless every single data point exists along the line of our prediction, then we’ll have some unexplained error, since models aren’t perfect

Note from Ravinder:

https://en.wikipedia.org/wiki/Coefficient_of_determination#/media/File:Coefficient_of_Determination.svg

The red squares represents baseline SSE, Blue squares represents SSE for your model. ESS = red - blue.

## tips dataset

1. Load the tips dataset from either pydataset or seaborn.

In [2]:
tips = sns.load_dataset("tips")

In [3]:
tips.head()

,total_bill,tip,sex,smoker,day,time,size
0,16.99,1.01,Female,No,Sun,Dinner,2
1,10.34,1.66,Male,No,Sun,Dinner,3
2,21.01,3.50,Male,No,Sun,Dinner,3
3,23.68,3.31,Male,No,Sun,Dinner,2
4,24.59,3.61,Female,No,Sun,Dinner,4


In [4]:
tips.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 244 entries, 0 to 243
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype   
---  ------      --------------  -----   
 0   total_bill  244 non-null    float64 
 1   tip         244 non-null    float64 
 2   sex         244 non-null    category
 3   smoker      244 non-null    category
 4   day         244 non-null    category
 5   time        244 non-null    category
 6   size        244 non-null    int64   
dtypes: category(4), float64(2), int64(1)
memory usage: 7.4 KB


2. Fit a linear regression model (ordinary least squares) and compute yhat, predictions of tip using total_bill.

    Here is some sample code to get you started:
    
    assuming X and y are already defined
    
    model = LinearRegression().fit(X, y)
    
    predictions = model.predict(X)

    Modify and add to the code above as necessary for it to work with the tips dataset.

In [50]:
lr1 = LinearRegression().fit(tips[["total_bill"]], tips["tip"])
yhat = lr1.predict(tips[["total_bill"]])

3. Plot the residuals for the linear regression model that you made.

In [51]:
tips["residual"] = tips["tip"] - yhat

In [8]:
tips.head()

,total_bill,tip,sex,smoker,day,time,size,residual
0,16.99,1.01,Female,No,Sun,Dinner,2,-1.694636
1,10.34,1.66,Male,No,Sun,Dinner,3,-0.346223
2,21.01,3.50,Male,No,Sun,Dinner,3,0.373165
3,23.68,3.31,Male,No,Sun,Dinner,2,-0.097250
4,24.59,3.61,Female,No,Sun,Dinner,4,0.107178


4. Calculate the sum of squared errors, explained sum of squares, total sum of squares, mean squared error, and root mean squared error for your model.

In [9]:
baseline = tips["tip"].mean()
tips['baseline'] = baseline
tips.head()

,total_bill,tip,sex,smoker,day,time,size,residual,baseline
0,16.99,1.01,Female,No,Sun,Dinner,2,-1.694636,2.998279
1,10.34,1.66,Male,No,Sun,Dinner,3,-0.346223,2.998279
2,21.01,3.50,Male,No,Sun,Dinner,3,0.373165,2.998279
3,23.68,3.31,Male,No,Sun,Dinner,2,-0.097250,2.998279
4,24.59,3.61,Female,No,Sun,Dinner,4,0.107178,2.998279


In [10]:
tips['baseline_residual'] = tips["tip"] - tips["baseline"]
tips.head()

,total_bill,tip,sex,smoker,day,time,size,residual,baseline,baseline_residual
0,16.99,1.01,Female,No,Sun,Dinner,2,-1.694636,2.998279,-1.988279
1,10.34,1.66,Male,No,Sun,Dinner,3,-0.346223,2.998279,-1.338279
2,21.01,3.50,Male,No,Sun,Dinner,3,0.373165,2.998279,0.501721
3,23.68,3.31,Male,No,Sun,Dinner,2,-0.097250,2.998279,0.311721
4,24.59,3.61,Female,No,Sun,Dinner,4,0.107178,2.998279,0.611721


In [11]:
# SSE (Sum of Squarred Errors)
tips['residual^2'] = tips['residual']**2
tips['baseline_residual^2'] = tips['baseline_residual']**2
tips.head()

,total_bill,tip,sex,smoker,day,time,size,residual,baseline,baseline_residual,residual^2,baseline_residual^2
0,16.99,1.01,Female,No,Sun,Dinner,2,-1.694636,2.998279,-1.988279,2.871792,3.953252
1,10.34,1.66,Male,No,Sun,Dinner,3,-0.346223,2.998279,-1.338279,0.119870,1.790990
2,21.01,3.50,Male,No,Sun,Dinner,3,0.373165,2.998279,0.501721,0.139252,0.251724
3,23.68,3.31,Male,No,Sun,Dinner,2,-0.097250,2.998279,0.311721,0.009458,0.097170
4,24.59,3.61,Female,No,Sun,Dinner,4,0.107178,2.998279,0.611721,0.011487,0.374203


In [12]:
SSE = tips['residual^2'].sum()
SSE_baseline = tips['baseline_residual^2'].sum()

print('SSE =', "{:.1f}".format(SSE))
print("SSE Baseline =", "{:.1f}".format(SSE_baseline))

SSE = 252.8
SSE Baseline = 465.2


In [14]:
# TSS = Total sum of squared error
# Note: TSS == SSE for baseline model (mean model)

TSS = SSE_baseline = tips['baseline_residual^2'].sum()

print('TSS =', "{:.1f}".format(TSS))

TSS = 465.2


In [18]:
# explained sum of squares
# ESS - Explained sum of squares ('Explained Error')
ESS = TSS - SSE

print('ESS =', "{:.1f}".format(ESS))

ESS = 212.4


In [19]:
# MSE (Mean Squared Error)
len(tips)
tips.shape[0]

244

In [20]:
MSE = SSE/len(tips)
MSE_baseline = SSE_baseline/len(tips)

print("MSE = ", "{:.1f}".format(MSE))
print("MSE baseline = ", "{:.1f}".format(MSE_baseline))

MSE =  1.0
MSE baseline =  1.9


In [21]:
# RMSE (root mean squared error)

RMSE = sqrt(MSE)
RMSE_baseline =  sqrt(MSE_baseline)

print("RMSE = ", "{:.1f}".format(RMSE))
print("RMSE baseline = ", "{:.1f}".format(RMSE_baseline))

RMSE =  1.0
RMSE baseline =  1.4


5. Calculate the sum of squared errors, mean squared error, and root mean squared error for the baseline model (i.e. a model that always predicts the average tip amount).

In [28]:
SSE_baseline = tips['baseline_residual^2'].sum()
MSE_baseline = SSE_baseline/len(tips)
RMSE_baseline =  sqrt(MSE_baseline)
print(f"baseline sse: {SSE_baseline}, baseline mse: {MSE_baseline}, baseline rmse: {RMSE_baseline}")


baseline sse: 465.2124770491804, baseline mse: 1.906608512496641, baseline rmse: 1.3807999538298954


6. Write python code that compares the sum of squared errors for your model against the sum of squared errors for the baseline model and outputs whether or not your model performs better than the baseline model.

In [32]:
prediction_sse = tips["residual^2"].sum()
print(f"prediction_sse: {prediction_sse}")
print(f"baseline_sse: {SSE_baseline}")
if prediction_sse < baseline_sse: print("prediction sse is better than baseline sse")

prediction_sse: 252.788743850776
baseline_sse: 465.2124770491804
prediction sse is better than baseline sse


7. What is the amount of variance explained in your model?

In [36]:
# R2: variance in y (target) explained by X (predictor); closer to 1 is better
r2 = ESS/TSS
r2
print(f"amount of variance: {r2}")

amount of variance: 0.45661658635167646


In [52]:
# can also calculated r2 using sklearn.metrics (r2_score)
r2_score(tips["tip"], yhat)

0.45661658635167657

8. Is your model better than the baseline model?

Yes, my model is better than the baseline.

9. Create a file named evaluate.py that contains the following functions.

- plot_residuals(y, yhat): creates a residual plot

- regression_errors(y, yhat): returns the following values

    - sum of squared errors (SSE)

    - explained sum of squares (ESS)

    - total sum of squares (TSS)

    - mean squared error (MSE)

    - root mean squared error (RMSE)

- baseline_mean_errors(y): computes the SSE, MSE, and RMSE for the baseline model


- better_than_baseline(y, yhat): returns true if your model performs better than the baseline, otherwise false

10. Load the mpg dataset and fit a model that predicts highway mileage based on engine displacement. Take a look at all the regression evaluation metrics, and determine whether this model is better than the baseline model. Use the functions from your evaluate.py to help accomplish this.

In [43]:
# acquire
mpg = sns.load_dataset("mpg")
mpg.head()

,mpg,cylinders,displacement,horsepower,weight,acceleration,model_year,origin,name
0,18.0,8,307.0,130.0,3504,12.0,70,usa,chevrolet chevelle malibu
1,15.0,8,350.0,165.0,3693,11.5,70,usa,buick skylark 320
2,18.0,8,318.0,150.0,3436,11.0,70,usa,plymouth satellite
3,16.0,8,304.0,150.0,3433,12.0,70,usa,amc rebel sst
4,17.0,8,302.0,140.0,3449,10.5,70,usa,ford torino


In [53]:
# fit
lr = LinearRegression().fit(mpg[["displacement"]], mpg["mpg"])
yhat = lr.predict(mpg[["displacement"]])

In [45]:
# establish the baseline
mpg["baseline"] = mpg["mpg"].mean()
mpg.head()

,mpg,cylinders,displacement,horsepower,weight,acceleration,model_year,origin,name,baseline
0,18.0,8,307.0,130.0,3504,12.0,70,usa,chevrolet chevelle malibu,23.514573
1,15.0,8,350.0,165.0,3693,11.5,70,usa,buick skylark 320,23.514573
2,18.0,8,318.0,150.0,3436,11.0,70,usa,plymouth satellite,23.514573
3,16.0,8,304.0,150.0,3433,12.0,70,usa,amc rebel sst,23.514573
4,17.0,8,302.0,140.0,3449,10.5,70,usa,ford torino,23.514573


In [54]:
mpg["residual"] = mpg["mpg"] - yhat

In [55]:
mpg['baseline_residual'] = mpg["mpg"] - mpg["baseline"]
mpg.head()

,mpg,cylinders,displacement,horsepower,weight,acceleration,model_year,origin,name,baseline,residual,baseline_residual,residual^2,baseline_residual^2
0,18.0,8,307.0,130.0,3504,12.0,70,usa,chevrolet chevelle malibu,23.514573,1.331948,-5.514573,1.774086,30.410514
1,15.0,8,350.0,165.0,3693,11.5,70,usa,buick skylark 320,23.514573,0.924092,-8.514573,0.853946,72.497951
2,18.0,8,318.0,150.0,3436,11.0,70,usa,plymouth satellite,23.514573,1.995055,-5.514573,3.980244,30.410514
3,16.0,8,304.0,150.0,3433,12.0,70,usa,amc rebel sst,23.514573,-0.848899,-7.514573,0.720629,56.468805
4,17.0,8,302.0,140.0,3449,10.5,70,usa,ford torino,23.514573,0.030536,-6.514573,0.000932,42.439660


In [49]:
mpg['residual^2'] = mpg['residual']**2
mpg['baseline_residual^2'] = mpg['baseline_residual']**2
mpg.head()

,mpg,cylinders,displacement,horsepower,weight,acceleration,model_year,origin,name,baseline,residual,baseline_residual,residual^2,baseline_residual^2
0,18.0,8,307.0,130.0,3504,12.0,70,usa,chevrolet chevelle malibu,23.514573,1.331948,-5.514573,1.774086,30.410514
1,15.0,8,350.0,165.0,3693,11.5,70,usa,buick skylark 320,23.514573,0.924092,-8.514573,0.853946,72.497951
2,18.0,8,318.0,150.0,3436,11.0,70,usa,plymouth satellite,23.514573,1.995055,-5.514573,3.980244,30.410514
3,16.0,8,304.0,150.0,3433,12.0,70,usa,amc rebel sst,23.514573,-0.848899,-7.514573,0.720629,56.468805
4,17.0,8,302.0,140.0,3449,10.5,70,usa,ford torino,23.514573,0.030536,-6.514573,0.000932,42.439660
